In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE169568"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE169568"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE169568.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE169568.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE169568.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"BeadChip microarray data of peripheral blood obtained from treatment-näive IBD patients and control individuals"
!Series_summary	"Comperhensive analysis of blood transcriptomes obtained from treatment-näive IBD patients."
!Series_overall_design	"Total RNA extracted from peripheral blood samples (n = 205) was reverse transcribed and biotin-labeled using the TargetAmp-Nano Labeling Kit for Illumina Expression BeadChip (Epicentre) according to the manufacturer’s protocol. The labeled antisense RNA was hybridized to Human HT-12 v4 BeadChip array (Illumina) following the standard producer’s hybridization protocol. The array imaging was performed on an iScan system (Illumina) according to the standard manufacturer’s protocol."
Sample Characteristics Dictionary:
{0: ['Sex: female', 'Sex: male'], 1: ['age: 20', 'age: 39', 'age: 56', 'age: 31', 'age: 22', 'age: 32', 'age: 30', 'age: 18', 'age: 60', 'age: 33', 'age: 27', 'age: 34', 'age: 57', 'age: 72', 'ag

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Check if this dataset likely contains gene expression data
# Based on the background information, this dataset contains BeadChip microarray data (Illumina Human HT-12 v4), 
# which is indeed gene expression data. So we set is_gene_available to True.
is_gene_available = True

# 2. Identify keys and conversion functions for trait, age, and gender data
# 2.1 Data Availability

# Trait - Crohn's Disease (key 2 contains diagnostic information)
trait_row = 2

# Age data (key 1 contains age information)
age_row = 1

# Gender data (key 0 contains sex information)
gender_row = 0

# 2.2 Data Type Conversion

def convert_trait(value):
    """
    Convert trait values to binary format:
    1 for Crohn's disease, 0 for controls (healthy or symptomatic)
    """
    if value is None:
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary based on diagnosis
    if "Crohn's disease" in value:
        return 1
    elif "Healthy control" in value or "Symptomatic control" in value or "Ulcerative colitis" in value:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age values to continuous format
    """
    if value is None:
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """
    Convert gender values to binary format:
    0 for female, 1 for male
    """
    if value is None:
        return None
    
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Determine trait data availability and save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the provided library function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data to the specified path
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM5209429': [0.0, 20.0, 0.0], 'GSM5209430': [0.0, 39.0, 1.0], 'GSM5209431': [0.0, 56.0, 0.0], 'GSM5209432': [0.0, 31.0, 0.0], 'GSM5209433': [1.0, 22.0, 1.0], 'GSM5209434': [0.0, 32.0, 1.0], 'GSM5209435': [0.0, 32.0, 0.0], 'GSM5209436': [0.0, 30.0, 0.0], 'GSM5209437': [0.0, 30.0, 1.0], 'GSM5209438': [0.0, 18.0, 0.0], 'GSM5209439': [0.0, 60.0, 0.0], 'GSM5209440': [0.0, 33.0, 1.0], 'GSM5209441': [0.0, 27.0, 0.0], 'GSM5209442': [0.0, 30.0, 1.0], 'GSM5209443': [0.0, 34.0, 0.0], 'GSM5209444': [0.0, 57.0, 1.0], 'GSM5209445': [0.0, 27.0, 1.0], 'GSM5209446': [0.0, 20.0, 0.0], 'GSM5209447': [0.0, 30.0, 0.0], 'GSM5209448': [1.0, 27.0, 1.0], 'GSM5209449': [0.0, 32.0, 1.0], 'GSM5209450': [0.0, 72.0, 0.0], 'GSM5209451': [1.0, 35.0, 0.0], 'GSM5209452': [0.0, 24.0, 0.0], 'GSM5209453': [1.0, 21.0, 1.0], 'GSM5209454': [0.0, 62.0, 1.0], 'GSM5209455': [1.0, 41.0, 0.0], 'GSM5209456': [0.0, 22.0, 0.0], 'GSM5209457': [0.0, 18.0, 0.0], 'GSM5209458': [0.0, 20.0, 1.0], 'GSM5209459': [1

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1651209', 'ILMN_1651229', 'ILMN_1651254', 'ILMN_1651262',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651315',
       'ILMN_1651336', 'ILMN_1651341', 'ILMN_1651343', 'ILMN_1651347',
       'ILMN_1651354', 'ILMN_1651358', 'ILMN_1651373', 'ILMN_1651378',
       'ILMN_1651385', 'ILMN_1651405', 'ILMN_1651415', 'ILMN_1651429'],
      dtype='object', name='ID')

Gene data dimensions: 11727 genes × 205 samples


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I can see they use the format ILMN_XXXXXXX
# These are Illumina BeadArray probe IDs, not human gene symbols
# Illumina probe IDs need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 3. Preview the gene annotation dataframe
print("Gene annotation dataframe column names:")
print(gene_annotation.columns)

# Preview the first few rows to understand the data structure
print("\nPreview of gene annotation data:")
annotation_preview = preview_df(gene_annotation, n=3)
print(annotation_preview)

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Gene annotation dataframe column names:
Index(['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene',
       'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID',
       'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id',
       'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE',
       'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband',
       'Definition', 'Ontology_Component', 'Ontology_Process',
       'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC'],
      dtype='object')

Preview of gene annotation data:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050'], 'Species': [nan, nan, nan], 'Source': [nan, nan, nan], 'Search_Key': [nan, nan, nan], 'Transcript': [nan, nan, nan], 'ILMN_Gene': [nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan], 'RefSeq_ID': [nan, nan, nan], 'Unigene_ID': [nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan], 'GI': [nan, nan, nan], 'Accession': [nan, nan, nan], 'Symbol': ['

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns in the gene annotation dataframe
# The 'ID' column in gene_annotation contains probe IDs that match gene_data.index
# The 'Symbol' column contains gene symbols that we want to map to

# 2. Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Ensure no empty gene symbols
gene_mapping = gene_mapping.dropna(subset=['Gene'])

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print dimensions of the processed gene expression data
print(f"\nMapped gene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Preview the first few gene symbols after mapping
print("\nFirst 10 gene symbols after mapping:")
print(gene_data.index[:10])



Mapped gene data dimensions: 11148 genes × 205 samples

First 10 gene symbols after mapping:
Index(['A2BP1', 'A2LD1', 'A4GNT', 'AAAS', 'AACS', 'AADACL1', 'AADACL4',
       'AAGAB', 'AAK1', 'AAMP'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 11039 genes × 205 samples


Normalized gene expression data saved to ../../output/preprocess/Crohns_Disease/gene_data/GSE169568.csv
Extracting clinical features from the original source...
Extracted clinical features preview:
{'GSM5209429': [0.0, 20.0, 0.0], 'GSM5209430': [0.0, 39.0, 1.0], 'GSM5209431': [0.0, 56.0, 0.0], 'GSM5209432': [0.0, 31.0, 0.0], 'GSM5209433': [1.0, 22.0, 1.0], 'GSM5209434': [0.0, 32.0, 1.0], 'GSM5209435': [0.0, 32.0, 0.0], 'GSM5209436': [0.0, 30.0, 0.0], 'GSM5209437': [0.0, 30.0, 1.0], 'GSM5209438': [0.0, 18.0, 0.0], 'GSM5209439': [0.0, 60.0, 0.0], 'GSM5209440': [0.0, 33.0, 1.0], 'GSM5209441': [0.0, 27.0, 0.0], 'GSM5209442': [0.0, 30.0, 1.0], 'GSM5209443': [0.0, 34.0, 0.0], 'GSM5209444': [0.0, 57.0, 1.0], 'GSM5209445': [0.0, 27.0, 1.0], 'GSM5209446': [0.0, 20.0, 0.0], 'GSM5209447': [0.0, 30.0, 0.0], 'GSM5209448': [1.0, 27.0, 1.0], 'GSM5209449': [0.0, 32.0, 1.0], 'GSM5209450': [0.0, 72.0, 0.0], 'GSM5209451': [1.0, 35.0, 0.0], 'GSM5209452': [0.0, 24.0, 0.0], 'GSM5209453': [1.0, 21.0, 1.0], '

Data shape after handling missing values: (205, 11042)

Checking for bias in feature variables:
For the feature 'Crohns_Disease', the least common label is '1.0' with 52 occurrences. This represents 25.37% of the dataset.
The distribution of the feature 'Crohns_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 24.0
  50% (Median): 34.0
  75%: 51.0
Min: 17.0
Max: 78.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 98 occurrences. This represents 47.80% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Crohns_Disease/GSE169568.csv
Final dataset shape: (205, 11042)
